In [133]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import unidecode
import pandas as pd
import numpy as np
import time

In [106]:
def lower_and_remove_diacritics(string):
    return unidecode.unidecode(string.lower())

In [105]:
def scrape_for_authors_and_albums(song_name, artist_name):
    global driver
    #search
    driver.find_element(By.CLASS_NAME, 'search-page').clear()
    search_box = driver.find_element(By.CLASS_NAME, 'search-page')
    song_name = lower_and_remove_diacritics(song_name)
    search_box.send_keys(song_name)
    search_box.send_keys(Keys.RETURN)
    time.sleep(3)
    
    
    #get results of search
    #search_result = driver.find_element(By.CLASS_NAME, 'search-result')
    try:
        song_results = driver.find_element(By.XPATH, f"//*[contains(text(), 'Pjesme')]/following-sibling::ul")
        song_results = song_results.find_elements(By.TAG_NAME, 'a')


        #go through search results
        for song in song_results:

            song.send_keys(Keys.CONTROL + Keys.RETURN)
            window_handles = driver.window_handles

            driver.switch_to.window(driver.window_handles[1])
            time.sleep(3)            

            if  check_performer(artist_name):
                subfields, authors = get_authors()
                albums = get_albums()
                driver.close()
                driver.switch_to.window(window_name=window_handles[0])
                return subfields, authors, albums
                
            else:
                driver.close()
                driver.switch_to.window(window_name=window_handles[0])
    
    except Exception as e: print(f"Problem with scraping authors and albums for song {song_name}")

    return False    


In [104]:
def check_performer(artist_name):
    authors = driver.find_elements(By.TAG_NAME, "h3")[1:]
    artist_name = lower_and_remove_diacritics(artist_name)
    for index, author in enumerate(authors):
        subfield = author.text.capitalize()
        if subfield == "Izvedba":
            siblings_of_current = driver.find_elements(By.XPATH, f"//*[contains(text(), '{subfield}')]/following-sibling::a")
            if index<len(authors)-1: 
                siblings_of_next = driver.find_elements(By.XPATH, f"//*[contains(text(), '{authors[index+1].text.capitalize()}')]/following-sibling::a")
                result = [r.text for r in siblings_of_current[0: len(siblings_of_current)-len(siblings_of_next)]]
            else: result = [r.text for r in siblings_of_current] 
            
    return True if len([r for r in result if artist_name in lower_and_remove_diacritics(r)])>0 else False


In [ ]:
def get_authors():
    all_fields = []
    all_authors = []
    #authors = driver.find_elements(By.TAG_NAME, "h3")[1:]
    authors = driver.find_elements(By.XPATH, f"//*[contains(text(), 'Autori')]/following-sibling::h3")
    for index, author in enumerate(authors):
        subfield = author.text.capitalize()
        siblings_of_current = driver.find_elements(By.XPATH, f"//*[contains(text(), '{subfield}')]/following-sibling::a")
        if len(siblings_of_current)>0:
            if index<len(authors)-1: 
                siblings_of_next = driver.find_elements(By.XPATH, f"//*[contains(text(), '{authors[index+1].text.capitalize()}')]/following-sibling::a")
                result = [r.text for r in siblings_of_current[0: len(siblings_of_current)-len(siblings_of_next)]]
            else: result = [r.text for r in siblings_of_current]
            all_fields.append(subfield)
            all_authors.append(str(result))
            
    return all_fields, all_authors


In [ ]:
def get_albums():
    albums = driver.find_elements(By.XPATH, f"//*[contains(text(), 'Albumi')]/following-sibling::div")
    #return [f"{album.text}-{album.find_element(By.TAG_NAME, 'a').text}" for album in albums]
    return [{album.text} for album in albums]      
    

In [99]:
#song_name = "Ako me ostaviš"

artist_name="Severina"


In [100]:
df = pd.read_csv(f"data/{artist_name}_cuspajz.csv")


In [ ]:
df_problematic = pd.DataFrame(columns =  ["Artist", "Song"])


12

In [134]:
ser = Service("C:/chromedriver_win32/chromedriver.exe")
op = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=ser, options=op)
driver.get("https://diskografija.com/")


#for row in df_problematic.iterrows():
for row in df.iterrows():
    artist_name = row[1].Artist#Diskografija
    song_name = row[1].Song
    try: song_name = song_name[0:song_name.index("(")]
    except:pass
    
    try: 
        subfields, authors, albums = scrape_for_authors_and_albums(song_name, artist_name)
    
        for subfield, author in zip(subfields, authors):
            df.loc[row[0], subfield] = author
            pass
        for i, album in enumerate(albums):
            df.loc[row[0], f"Album_{i+1}"] = album
            pass
        print(f"Succesfully found information for {song_name} from {artist_name}")
    
    except Exception as e: 
        df_problematic.loc[row[0], "Artist"] = artist_name
        df_problematic.loc[row[0], "Song"] = song_name
        print(f"Problem with finding info for {song_name} from {artist_name}")
        print(e)
    print("#######")
    time.sleep(3)
    


Problem with scraping authors and albums for song adam i seva


NameError: name 'df_problematic' is not defined

In [ ]:
df.to_csv(f"data/{artist_name}_cuspajz_diskografija.csv", index=False)
df_problematic.to_csv(f"data/{artist_name}_problematic.csv", index=False)


In [135]:
##Rerun problematic songs, 

ser = Service("C:/chromedriver_win32/chromedriver.exe")
op = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=ser, options=op)
driver.get("https://diskografija.com/")


for row in df_problematic.iterrows():
    artist_name = row[1].Artist#Diskografija
    song_name = row[1].Song
    try: song_name = song_name[0:song_name.index("(")]
    except:pass

    try: 
        subfields, authors, albums = scrape_for_authors_and_albums(song_name, artist_name)
    
        for subfield, author in zip(subfields, authors):
            df.loc[row[0], subfield] = author
            pass
        for i, album in enumerate(albums):
            df.loc[row[0], f"Album_{i+1}"] = album
            pass
        print(f"Succesfully found information for {song_name} from {artist_name}")
        df_problematic = df_problematic.drop(row[0])
    
    except Exception as e: 
        df_problematic.loc[row[0], "Artist"] = artist_name
        df_problematic.loc[row[0], "Song"] = song_name
        print(f"Problem with finding info for {song_name} from {artist_name}")
        print(e)
    print("#######")
    time.sleep(3)
    


KeyboardInterrupt: 

In [106]:
df.to_csv(f"data/{artist_name}_cuspajz_diskografija.csv", index=False)
df_problematic.to_csv(f"data/{artist_name}_problematic.csv", index=False)


In [116]:
def open_google():
    ser = Service("C:/chromedriver_win32/chromedriver.exe")
    op = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=ser, options=op)
    driver.get("https://google.hr")
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//*[contains(text(), 'Aceito')]"))).click()
    time.sleep(0.5)
    driver.find_element(By.XPATH, "//*[contains(text(), 'English')]").click()
    time.sleep(0.5)

In [111]:
def search_google(search_query):
     #search
    driver.find_element(By.NAME, 'q').clear()
    search_box = driver.find_element(By.NAME, 'q')
    search_box.send_keys(search_query)
    search_box.send_keys(Keys.RETURN)
    time.sleep(3)

In [183]:
def get_lyricstranslate():
    

    driver.find_element(By.PARTIAL_LINK_TEXT, 'lyricstranslate.com').send_keys(Keys.CONTROL + Keys.RETURN)
    time.sleep(3)
    window_handles = driver.window_handles
    driver.switch_to.window(window_name=window_handles[1])    
    time.sleep(3)
    try: driver.find_element(By.XPATH, "//*[contains(text(), 'AGREE')]").click()
    except: pass
    time.sleep(3)
    try: driver.find_element(By.PARTIAL_LINK_TEXT, 'Click to see the original lyrics').click()
    except: pass
    time.sleep(3)
    original_lyrics = driver.find_element(By.CLASS_NAME, 'song-node-text')
    par_class = original_lyrics.find_elements(By.CLASS_NAME, 'par')
    lyrics = ". ".join([t.text for t in par_class])
    lyrics = lyrics.replace("*", "")
        
    try:
        copyright = original_lyrics.find_element(By.CLASS_NAME, 'copyrighttext').text.split("\n")
        writer_info = [c for c in copyright if "Writer" in c][0]
        if len(writer_info)>0:
            writer = writer_info.split(": ")[1]
        driver.close()
        driver.switch_to.window(window_name=window_handles[0])
    except: 
        print("Couldn't find writer info")
        writer = np.nan
        driver.close()
        driver.switch_to.window(window_name=window_handles[0])
    
    
    return lyrics, writer

In [182]:
ser = Service("C:/chromedriver_win32/chromedriver.exe")
op = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=ser, options=op)
driver.get("https://google.hr")
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//*[contains(text(), 'Aceito')]"))).click()
time.sleep(0.5)
driver.find_element(By.XPATH, "//*[contains(text(), 'English')]").click()
time.sleep(0.5)

df_new = df.copy()[0:5]
for row in df_new.iterrows():
    time.sleep(1)
    artist_name = row[1].Artist#Diskografija
    song_name = row[1].Song
    try: song_name = song_name[0:song_name.index("(")]
    except:pass
    
    try:
        search_google(lower_and_remove_diacritics(artist_name+" "+song_name+ " lyricstranslate"))
        lyrics, writer = get_lyricstranslate()
        print(writer, lyrics)
    
    except Exception as e:
        print(f"Problem with finding info for {song_name} from {artist_name}")
        print(e)
    print("#######")
    time.sleep(3)
    


ERRORRR
DONE
#######
ERRORRR
Couldn't find writer info
#######
ERRORRR
Couldn't find writer info
#######
ERRORRR
DONE
#######
Problem with finding info for Bambola from Severina
Message: no such element: Unable to locate element: {"method":"partial link text","selector":"lyricstranslate.com"}
  (Session info: chrome=98.0.4758.102)
Stacktrace:
Backtrace:
	Ordinal0 [0x00F569A3+2582947]
	Ordinal0 [0x00EEA6D1+2139857]
	Ordinal0 [0x00DE3A98+1063576]
	Ordinal0 [0x00E0FF3E+1244990]
	Ordinal0 [0x00E1013B+1245499]
	Ordinal0 [0x00E39F52+1417042]
	Ordinal0 [0x00E28594+1344916]
	Ordinal0 [0x00E3834A+1409866]
	Ordinal0 [0x00E28366+1344358]
	Ordinal0 [0x00E05176+1200502]
	Ordinal0 [0x00E06066+1204326]
	GetHandleVerifier [0x010FBE02+1675858]
	GetHandleVerifier [0x011B036C+2414524]
	GetHandleVerifier [0x00FEBB01+560977]
	GetHandleVerifier [0x00FEA8D3+556323]
	Ordinal0 [0x00EF020E+2163214]
	Ordinal0 [0x00EF5078+2183288]
	Ordinal0 [0x00EF51C0+2183616]
	Ordinal0 [0x00EFEE1C+2223644]
	BaseThreadInitThunk 

In [138]:
lyrics

"U mojoj ulici jutro donose ptice\ni osmijehe na svako lice\ndjeca za loptom trče i svi se smiju\ni nešto mi dovikuju. Dok šetam počinje kiša, ugledam tebe\nsami ja i ti, gdje se sakriti\npuni su izlozi kišobrana i parfema\ni nešto se dobro sprema. Ref.\nAj, aj, aj, aj\nsve je cool i sve je fino\naj, aj, aj, aj\nbaš se dobro osjećam\ns neba ljubav, sami ti i ja\naj, aj, aj, aj\nsve je cool i sve je fino. U mojoj ulici kafići su puni ljudi\nnovine i kava, svi su pomalo ludi\nna petom je katu netko pustio latino\ni to je u redu i baš nam je fino. U svakoj kapi kiše\nljubav čeka na nas\ni kad je san il' java\nja ti čujem glas. Ref."